In [69]:
import pyspark
sc = pyspark.SparkContext.getOrCreate()
#(idusuario,nickname, pais, fecha_alta)
usuarios = [(0, "aaa1", "argentina", "2018-10-10"),
           (1, "aaa2", "argentina", "2018-10-12"),
           (2, "aaa3", "argentina", "2018-10-10"),
           (3, "aaa4", "argentina", "2018-10-13"),
           (4, "aaa5", "brasil", "2018-10-10"),
           (5, "aaa6", "brasil", "2018-10-10"),
           (6, "aaa7", "argentina", "2018-10-20"),
           (7, "aaa8", "argentina", "2018-11-1"),
           (8, "aaa9", "argentina", "2018-12-10"),
           (9, "aaa10", "argentina", "2018-11-10"),
           (10, "aaa11", "brasil", "2018-10-15"),
           (11, "aaa12", "argentina", "2018-10-1"),
           (12, "aaa13", "argentina", "2018-10-2"),
           (13, "aaa14", "brasil", "2018-10-3"),
           (14, "aaa15", "argentina", "2018-10-9")]
#(idpublic,idusuario, idfoto, comentario, fecha)
publicaciones = [(0,0, 12, "#feliz #triste #casual #hola", "2018-10-9"),
                (1,0, 12, "#feliz #triste ", "2018-10-9"),
                (2,0, 12, "#feliz #triste #hola", "2018-10-9"),
                (3,1, 12, "#feliz #casual #hola", "2018-10-9"),
                (4,1, 12, "#feliz #triste #casual #hola", "2018-10-9"),
                (5,1, 12, "#feliz ", "2018-10-9"),
                (0,2, 12, " ", "2018-10-9"),
                (0, 2, 12, "#asd", "2018-10-9"),
                (0,3,  12, "#aaaaaaaaaaa", "2018-10-9"),
                (0,3, 12, "#feliz #triste #casual #hola", "2018-10-9"),
                (0,4,  12, "#feliz #triste #casual #hola", "2018-10-9"),
                (0,4,  12, "#asaaaa", "2018-10-9"),
                (0,6,  12, "#aazxca", "2018-10-9"),
                (0,5,  12, "#feliz #triste ", "2018-10-9"),
                (0,7,  12, "#hola", "2018-10-9"),
                (0,9,  12, "casual #hola", "2018-10-9"),
                (0,11, 12, "#feliz #triste #casual #hola", "2018-10-9"),
                (0,12, 12, "hola hola hola hola chau chau #hola", "2018-10-9"),
                (0,14, 12, "#feliz triste", "2018-10-9"),
                (0,13, 12, "#feliz #triste #casual #hola", "2018-10-9"),
                (0,13, 12, "#feliz #triste #casual #hola", "2018-10-9"),]

rdd_usuarios = sc.parallelize(usuarios)
rdd_publicaciones = sc.parallelize(publicaciones)

In [110]:
#hashtags que más aparecen por usuarios de argentina
userComments = rdd_publicaciones.map(lambda x : (x[1], ([x[3]]))).reduceByKey(lambda x,y : x + y)
argentinaUsers = rdd_usuarios.filter(lambda x : x[2] == "argentina")
wordsListByUser = argentinaUsers.map(lambda x : (x[0], x[1])).join(userComments).map(lambda x : x[1][1])
wordsListByUser.collect()

[['#feliz #triste #casual #hola', '#feliz #triste ', '#feliz #triste #hola'],
 ['#feliz #casual #hola', '#feliz #triste #casual #hola', '#feliz '],
 [' ', '#asd'],
 ['#aaaaaaaaaaa', '#feliz #triste #casual #hola'],
 ['#aazxca'],
 ['#hola'],
 ['casual #hola'],
 ['#feliz #triste #casual #hola'],
 ['hola hola hola hola chau chau #hola'],
 ['#feliz triste']]

In [113]:
stringList = wordsListByUser.flatMap(lambda x : map(lambda y : y, x))

In [122]:
stringList.flatMap(lambda x : x.split()).filter(lambda x : x.startswith("#")).map(lambda x : (x, 1)).reduceByKey(lambda x,y : x + y).takeOrdered(10, key=lambda x : -x[1])

[('#hola', 9),
 ('#feliz', 9),
 ('#triste', 6),
 ('#casual', 5),
 ('#aaaaaaaaaaa', 1),
 ('#aazxca', 1),
 ('#asd', 1)]

In [153]:
wordsForUsers = userComments.flatMap(lambda x : map(lambda y : (x[0], y.split()), x[1])).reduceByKey(lambda x,y : x + y)
wordAndUser = wordsForUsers.flatMap(lambda x : map(lambda y : (y , x[0]), x[1])).filter(lambda x : x[0].startswith("#"))
wordAndUser.join(wordAndUser).distinct().collect()

[('#hola', (12, 3)),
 ('#hola', (1, 4)),
 ('#hola', (4, 3)),
 ('#hola', (9, 4)),
 ('#casual', (1, 1)),
 ('#casual', (3, 3)),
 ('#casual', (3, 11)),
 ('#triste', (1, 4)),
 ('#triste', (4, 11)),
 ('#feliz', (1, 1)),
 ('#feliz', (13, 13)),
 ('#feliz', (3, 3)),
 ('#feliz', (3, 11)),
 ('#feliz', (11, 11)),
 ('#hola', (13, 1)),
 ('#hola', (7, 11)),
 ('#hola', (9, 13)),
 ('#casual', (1, 0)),
 ('#casual', (13, 4)),
 ('#aaaaaaaaaaa', (3, 3)),
 ('#feliz', (1, 0)),
 ('#hola', (0, 9)),
 ('#casual', (1, 3)),
 ('#triste', (0, 1)),
 ('#triste', (4, 5)),
 ('#triste', (11, 0)),
 ('#feliz', (0, 0)),
 ('#feliz', (1, 3)),
 ('#feliz', (1, 11)),
 ('#hola', (4, 4)),
 ('#hola', (9, 11)),
 ('#casual', (0, 1)),
 ('#feliz', (14, 3)),
 ('#feliz', (14, 11)),
 ('#feliz', (3, 0)),
 ('#feliz', (4, 5)),
 ('#hola', (1, 0)),
 ('#hola', (13, 12)),
 ('#asaaaa', (4, 4)),
 ('#triste', (5, 4)),
 ('#feliz', (0, 14)),
 ('#hola', (1, 1)),
 ('#hola', (3, 11)),
 ('#hola', (9, 1)),
 ('#casual', (1, 4)),
 ('#casual', (4, 11)),
 ('#